In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim

In [ ]:
import nltk
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

In [ ]:
RANDOM_SEED = 123
DEVICE = "cuda"
df = pd.read_csv("/content/drive/My Drive/data/lyrics1.csv", delimiter = ",", dtype = {'lyrics':str})

In [ ]:
df['ranker_genre'] = np.where((df['ranker_genre'] == 'screamo')|(df['ranker_genre'] == 'punk rock')|(df['ranker_genre'] == 'heavy metal'),'alt rock',df['ranker_genre'])
df['ranker_genre'] = np.where((df['ranker_genre']== 'rythm and blues'),'pop',df['ranker_genre'])
group = ['song', 'year', 'album', 'genre', 'artist', 'ranker_genre']
lyrics_by_song = df.sort_values(group).groupby(group).lyric.apply(' '.join).apply(lambda x: x.lower()).reset_index(name='lyric')
lyrics_by_song["lyric"] = lyrics_by_song['lyric'].str.replace(r'[^\w\s]','')

In [ ]:
fig = plt.figure(figsize=(8,6))
lyrics_by_song.groupby('ranker_genre').lyric.count().plot.bar(ylim=0)
plt.show()

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;|\n]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
lyrics = lyrics_by_song["lyric"]

In [ ]:
def clean_text(text):
    text = BeautifulSoup(text, "lxml").text 
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text) 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    return text

In [ ]:
lyrics = lyrics.apply(clean_text)
lyrics_by_song["lyric"]=lyrics

In [ ]:
X = lyrics_by_song.lyric
y = lyrics_by_song.ranker_genre
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', LogisticRegression(n_jobs=1, multi_class = "multinomial",  C=100000, solver = "sag", class_weight="balanced")),])

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)

In [ ]:
my_tags = ["country","hip hop","alt rock","pop","rhythm and blues"]

In [ ]:
print(accuracy_score(y_pred, y_test))

0.8115514559982839
